In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-data/equip_failures_test_set.csv
/kaggle/input/training1-dataset/equip_failures_training_set.csv
/kaggle/input/sample-submission/sample_submission.csv


In [2]:
submit = 1
X_train = pd.read_csv("/kaggle/input/training1-dataset/equip_failures_training_set.csv")
X_test = pd.read_csv("/kaggle/input/test-data/equip_failures_test_set.csv")
X_test_id = X_test.id
y_train = X_train["target"]
X_train.drop(columns=["id",'target'], axis=1, inplace=True)
X_test.drop(columns=["id"], axis=1, inplace=True)

In [3]:
# num_cols = [col for col in X_train.columns if X_train[col].dtype == "int64" or X_train[col].dtype == "float64"]
# cat_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

cols = X_train.columns

from sklearn.model_selection import train_test_split
if not submit:
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20, random_state=1)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
for col in cols:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')

In [5]:
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 500)
print(X_train.dtypes)
# print(len(num_cols))
# print(len(cat_cols))
X_train.head(2)
# print(X_train.isnull())
print(X_train.isnull().sum(axis=0).head(169))


sensor1_measure               int64
sensor2_measure             float64
sensor3_measure             float64
sensor4_measure             float64
sensor5_measure             float64
                             ...   
sensor105_histogram_bin7    float64
sensor105_histogram_bin8    float64
sensor105_histogram_bin9    float64
sensor106_measure           float64
sensor107_measure           float64
Length: 170, dtype: object
sensor1_measure                 0
sensor2_measure             46329
sensor3_measure              3335
sensor4_measure             14861
sensor5_measure              2500
                            ...  
sensor105_histogram_bin6      671
sensor105_histogram_bin7      671
sensor105_histogram_bin8      671
sensor105_histogram_bin9      671
sensor106_measure            2724
Length: 169, dtype: int64


In [6]:
# print(X_train.dtypes)
# print(len(num_cols))
# print(len(cat_cols))

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [8]:
num_transformer = Pipeline(steps=[
    ("num_imputer", SimpleImputer(strategy="mean"))
])

# cat_transformer = Pipeline(steps=[
#     ("cat_imputer", SimpleImputer(strategy="most_frequent")),
#     ("cat_one_hot_encoding",OneHotEncoder(handle_unknown="ignore"))
# ])

In [9]:
col_transformer = ColumnTransformer([
    ("num_transformer", num_transformer, cols),
#     ("cat_transformer", cat_transformer, cat_cols) 
])

In [10]:
pre_proc_pipeline = Pipeline(steps=[
("preprocessor", col_transformer)
])

X_train = pre_proc_pipeline.fit_transform(X_train)
# if not submit:
#     X_valid = pre_proc_pipeline.transform(X_valid)

In [11]:
model = RandomForestRegressor()

from sklearn.model_selection import GridSearchCV

params_grid = {
    'n_estimators' : [500],
    'max_depth' : [5],
}

In [12]:
grid = GridSearchCV(model, params_grid, cv=5, scoring="f1")

In [13]:
grid.fit(X_train, y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 282, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 1146, in f1_score
    return fbeta_score(
  File "/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py", line 1287, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/opt/conda/l

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [5], 'n_estimators': [500]},
             scoring='f1')

In [14]:
print(grid.best_params_)
print(grid.cv_results_)

{'max_depth': 5, 'n_estimators': 500}
{'mean_fit_time': array([1138.75730219]), 'std_fit_time': array([19.45310531]), 'mean_score_time': array([0.29330282]), 'std_score_time': array([0.02824763]), 'param_max_depth': masked_array(data=[5],
             mask=[False],
       fill_value='?',
            dtype=object), 'param_n_estimators': masked_array(data=[500],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'max_depth': 5, 'n_estimators': 500}], 'split0_test_score': array([nan]), 'split1_test_score': array([nan]), 'split2_test_score': array([nan]), 'split3_test_score': array([nan]), 'split4_test_score': array([nan]), 'mean_test_score': array([nan]), 'std_test_score': array([nan]), 'rank_test_score': array([1], dtype=int32)}


In [15]:
if not submit:
    val_mse = np.sqrt(mean_squared_error(model.predict(X_valid), y_valid))
    print(f" val_mse : {val_mse}\n log val_mse : {np.log(val_mse)}")
    
    train_mse = np.sqrt(mean_squared_error(model.predict(X_train), y_train))
    print(f" train_mse : {train_mse}\n log train_mse : {np.log(train_mse)}")
    
else:
    train_mse = np.sqrt(mean_squared_error(grid.predict(X_train), y_train))
    print(f" train_mse : {train_mse}\n log train_mse : {np.log(train_mse)}")

 train_mse : 0.07391483218104362
 log train_mse : -2.604841765063324


In [16]:
# X_test = pre_proc_pipeline.transform(X_test)
# predictions = grid.predict(X_test)

In [17]:
cols = X_test.columns
for col in cols:
    X_test[col] = pd.to_numeric(X_test[col], errors='coerce')

In [18]:
print(len(cols))

170


In [19]:
print(X_test.dtypes)

sensor1_measure               int64
sensor2_measure             float64
sensor3_measure             float64
sensor4_measure             float64
sensor5_measure             float64
                             ...   
sensor105_histogram_bin7    float64
sensor105_histogram_bin8    float64
sensor105_histogram_bin9    float64
sensor106_measure           float64
sensor107_measure           float64
Length: 170, dtype: object


In [20]:
X_test = pre_proc_pipeline.transform(X_test)

In [22]:
predictions = grid.predict(X_test)

In [28]:
output = pd.DataFrame({"id":X_test_id, "target":predictions})

In [29]:
output.to_csv("/kaggle/working/submission.csv", index=False)